# Einleitung PINA
Source: [PINA-Introduction](https://mathlab.github.io/PINA/_rst/tutorials/tutorial1/tutorial.html) <br>

In diesem Tutorial werden wir nach dem Standard -API -Verfahren einen typischen Anwendungsfall von Pina für ein Spielzeugproblem demonstrieren.<br>

Insbesondere zielt das Tutorial darauf ab, die folgenden Themen einzuführen:<br>

- Erklären, wie man <strong>PINA</strong> Probleme aufbaut,
- Zeigen, wie Daten für das `PINN` Training generiert werden

Dies sind die beiden Hauptschritte, die vor Beginn der Modellierungsoptimierung benötigt werden (Model und Löser und Zug). Wir werden jeden Schritt ausführlich zeigen und am Ende ein einfaches Problem mit der normalen Differentialgleichung (ODE) unter Verwendung des Pinn -Solvers lösen.

## Variablendeklaration
- `SpatialProblem`: Differentialgleichung mit Spatialen Variablen `spatial_domain`<br>
- `TimeDependentProblem`: Zeitabhängige Differentialgleichung mit zeitabhängigen Variablen `temporal_domain`<br>
- `ParametricProblem`: Parametrisierte Differentialgleichung mit parametrischen Variablen`parameter_domain`<br>
- `AbstractProblem`: Sonstige <strong>PINA</strong> Problemerben<br><br>

## Definitionen
- `Residual` : Differenz zwischen der linken und rechten Seite einer Differentialgleichung<br>
- `truth_solution` : Funktion, die die wahre Lösung der Differentialgleichung berechnet. <br>
$\rightarrow$ Hilfreich um die Vergleichbarkeit darzustellen<br>

## Problemdefinition
<strong>Annahme:</strong>
$$
\begin{cases}
  \frac{d}{dx}u(x) &= u(x) \quad \text{mit} \quad u(0) \in 1\\
  u(x=0) &=1\\
\end{cases}
$$
<strong>Analytische Lösung:</strong>
$$
u(x) = e^x \quad x \in (0,1)
$$

In [2]:
from pina.problem import SpatialProblem
from pina.operators import grad
from pina import Condition
from pina.geometry import CartesianDomain
from pina.equation import Equation, FixedValue

import torch

class SimpleODE(SpatialProblem): # Problem Class

    output_variables = ['u']
    spatial_domain = CartesianDomain({'x': [0,1]})

    # defining the ode equation
    def ode_equation(input_, output_):

        # computing the derivative
        u_x = grad(output_, input_, components=['u'], d=['x'])

        #extracting the u input variable
        u = output_.extract(['u'])

        # calculate the residual and return it
        return u_x - u

    # conditions to hold
    conditions = {
        # We fix initial condition to value 1
        'x0': Condition(location=CartesianDomain({'x': 0.}), equation=FixedValue(1)),
        # We wrap the python equation using Equation
        'D': Condition(location=CartesianDomain({'x': [0,1]}), equation=Equation(ode_equation)),
    }

    # sample points (see below)labels
    input_pts = None

    # defining the true solution
    def truth_solution(self, pts):
        return torch.exp(pts.extract(['x']))

problem = SimpleODE()

## Datengenerierung (Trainingsdaten)
Daten für das Training können in Form von direkten numerischen Simulationsergebnissen oder Punkten in den Domänen vorliegen. Im Falle von unüberwachtem Lernen benötigen wir nur die Kollokationspunkte für das Training, d.h. Punkte, an denen wir das neuronale Netzwerk auswerten möchten.

### Erläuterung
- <strong>Direkte numerische Simulationsergebnisse:</strong> Dies sind Ergebnisse, die durch numerische Simulationen generiert wurden.
- <strong>Manuell definierte Punkte:</strong> Diese können unabhängig von Simulationsergebnissen festgelegt werden, um das Modell an bestimmten Stellen zu trainieren oder zu evaluieren.

### Beispiel
Drei Beispiele der Datengenerierung sind im folgenden Code dargestellt.

In [ ]:
# sampling 20 points in [0, 1] through discretization in all locations
problem.discretise_domain(n=20, mode='grid', variables=['x'], locations='all')

# sampling 20 points in (0, 1) through latin hypercube sampling in D, and 1 point in x0
problem.discretise_domain(n=20, mode='latin', variables=['x'], locations=['D'])
problem.discretise_domain(n=1, mode='random', variables=['x'], locations=['x0'])

# sampling 20 points in (0, 1) randomly
problem.discretise_domain(n=20, mode='random', variables=['x'])

## Latin Hypercube Sampling (LHS)
Latin Hypercube Sampling ist eine Methode zur gleichmäßigen und effizienten Stichprobenentnahme aus einem mehrdimensionalen Raum. Dabei wird jede Dimension in gleich große Intervalle unterteilt, aus denen je ein Wert zufällig gewählt wird, sodass eine möglichst gleichmäßige Verteilung der Punkte entsteht. In PINA wird LHS genutzt, um die Eingabepunkte für das neuronale Netz optimal über den Definitionsbereich der Differentialgleichung zu verteilen und die Lösungsgüte zu verbessern.

### Beispiel

In [3]:
# sampling for training
problem.discretise_domain(1, 'random', locations=['x0'])
problem.discretise_domain(20, 'lh', locations=['D'])

### Abrufen der Daten
Mithilfe von `dict` `input_pts`

In [4]:
print('Input points:', problem.input_pts)
print('Input points labels:', problem.input_pts['D'].labels)

Input points: {'x0': LabelTensor([[[0.]]]), 'D': LabelTensor([[[0.5957]],
             [[0.9689]],
             [[0.4208]],
             [[0.2270]],
             [[0.0843]],
             [[0.6168]],
             [[0.3718]],
             [[0.6594]],
             [[0.9442]],
             [[0.8778]],
             [[0.8382]],
             [[0.1483]],
             [[0.3195]],
             [[0.2575]],
             [[0.1910]],
             [[0.0387]],
             [[0.7852]],
             [[0.7163]],
             [[0.4979]],
             [[0.5414]]])}
Input points labels: ['x']


## Visualisierung
Mithilfe von `.plot_samples` `Plotter`

In [ ]:
from pina import Plotter

pl = Plotter()
pl.plot_samples(problem=problem)

## Beispiel Training
Anhand eines sehr einfachen Beispiels

In [ ]:
from pina import Trainer
from pina.solvers import PINN
from pina.model import FeedForward
from pina.callbacks import MetricTracker
import torch

# build the model
model = FeedForward(
    layers=[10, 10],
    func=torch.nn.Tanh,
    output_dimensions=len(problem.output_variables),
    input_dimensions=len(problem.input_variables)
)

# create the PINN object
pinn = PINN(problem, model)

# create the trainer
trainer = Trainer(solver=pinn,
                  max_epochs=1500,
                  callbacks=[MetricTracker()],
                  accelerator='cpu',
                  enable_model_summary=False) # we train on CPU and avoid model summary at beginning of training (optional)

# train
trainer.train()

## Ergebnisse der Modellierung
`trainer.logged_metrics` enthält den finalen Verlust und weitere Metriken, die während des Trainings protokolliert wurden. Standardmäßig wird der Verlust als `mean square error` gespeichert.

In [ ]:
# inspecting final loss
trainer.logged_metrics

## Visualisierung der Ergebnisse

In [ ]:
# plotting the solution
pl.plot(solver=pinn)

## Visualisierung der Loss-Kurve

In [ ]:
pl.plot_loss(trainer=trainer, label='mean_loss', logy=True)

### Erkenntnis
- Der Verlust hat nicht das Minimum erreicht. Man könnte länger trainieren.

## Weitere Optimierungsmöglichkeiten
1. Das Netzwerk länger Trainieren mit unterschiedlichen Layergrößen und Verbesserung der finalen Genauigkeit
2. Das netzwerk mit anderen Modeltypen testen
3. GPU Training und Speed Benchmarking